In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage.tranform import resize
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing import image

sns.set_style('darkgrid')
#create an empty DataFrame
df = pd.DataFrame(columns=['path','label'])
fire_cnt, non_fire_cnt = 0, 0
#loop over fire images and label them 1
for dirname, _, filenames in os.walk('/content/drive/MyDrive/fire_dataset/fire_images'):
    for filename in filenames:
        fire_cnt+=1
        #print(os.path.join(dirname, filename))
        df = df.append(pd.DataFrame([[os.path.join(dirname, filename),'fire']],columns=['path','label']))

#loop over non fire images and label them 0
for dirname, _, filenames in os.walk('/content/drive/MyDrive/fire_dataset/non_fire_images'):
    for filename in filenames:
        non_fire_cnt+=1
        df = df.append(pd.DataFrame([[os.path.join(dirname, filename),'non_fire']],columns=['path','label']))
        #print(os.path.join(dirname, filename))

#shuffle the dataset to redistribute the labels
df = df.sample(frac=1).reset_index(drop=True)
df.head(10)

fig = px.scatter(data_frame = df,x=df.index,y='label',color='label',title='Distribution of fire and non-fire images along the length of the dataframe')
fig.update_traces(marker_size=2)

fig = make_subplots(rows=1, cols=2, specs=[[{"type": "xy"}, {"type": "pie"}]])

fig.add_trace(go.Bar(x =df['label'].value_counts().index,y=df['label'].value_counts().to_numpy(),marker_color=['darkorange','green'],showlegend=False),row=1,col=1)

fig.add_trace(go.Pie(
     values=df['label'].value_counts().to_numpy(),
     labels=df['label'].value_counts().index,
    marker=dict(colors=['darkorange','green'])),
    row=1, col=2)

label = 'fire' #label for images with fire
label_negative = 'non_fire'
non_data = df[df['label']== label_negative]
df.describe()
data = df[df['label'] == label]
sns.set_style('dark')

#display the fire images
pics = 6 #set the number of pics
fig,ax = plt.subplots(int(pics//2),2,figsize=(15,15))
plt.suptitle('Images with Fire')
ax = ax.ravel() #flatten the subplots array
for i in range(6):
    path = data.sample(1).loc[:,'path'].to_numpy()[0]
    img = image.load_img(path)
    img = image.img_to_array(img)/255
    ax[i].imshow(img)
    ax[i].axes.xaxis.set_visible(False)
    ax[i].axes.yaxis.set_visible(False)

#display the non-fire images
pics = 6
fig, ax = plt.subplots(3, 2, figsize = (15,15))
plt.suptitle('Images WITHOUT Fire')
ax = ax.ravel()
for i in range(6):
    path = non_data.sample(1).loc[:, 'path'].to_numpy()[0]
    img = image.load_img(path)
    img = image.img_to_array(img)/255 #0~1
    ax[i].imshow(img)
    ax[i].axes.xaxis.set_visible(False)
    ax[i].axes.yaxis.set_visible(False)
    
#visualize the distribution of their shapes (images)
def shaper(row):
    print(row)
    shape = image.load_img(row['path']).size
    print(shape)
    row['height'] = shape[1]
    row['width'] = shape[0]
    return row
df = df.apply(shaper,axis=1)
df.head(5)

sns.set_style('darkgrid')
fig,(ax1,ax2,ax3) = plt.subplots(1,3,gridspec_kw={'width_ratios': [3,0.5,0.5]},figsize=(15,10))
sns.kdeplot(data=df.drop(columns=['path','label']),ax=ax1,legend=True)
sns.boxplot(data=df,y='height',ax=ax2,color='skyblue')
sns.boxplot(data=df,y='width',ax=ax3,color='orange')
plt.suptitle('Distribution of image shapes')
ax3.set_ylim(0,7000)
ax2.set_ylim(0,7000)
plt.tight_layout()

from keras.preprocessing.image import ImageDataGenerator
generator = ImageDataGenerator(
    rotation_range= 20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range = 2,
    zoom_range=0.2,
    rescale = 1/255,
    validation_split=0.2,
)

train_gen = generator.flow_from_dataframe(df,x_col='path',y_col='label',images_size=(256,256),class_mode='binary',subset='training')
val_gen = generator.flow_from_dataframe(df,x_col='path',y_col='label',images_size=(256,256),class_mode='binary',subset='validation')
#augments the input

class_indices = {} 
for key in train_gen.class_indices.keys():
    class_indices[train_gen.class_indices[key]] = key
#result : 0 for fire, 1 for non-fire    
print(class_indices)

#Now, a CNN model that trains parameters from scratch
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense
model = keras.models.Sequential([
    keras.layers.Conv2D(filters = 32, kernel_size = (2,2), activation = 'relu',
                        input_shape = (256, 256, 3)),
    keras.layers.MaxPool2D(),
    keras.layers.Conv2D(filters = 64, kernel_size = (2,2), activation = 'relu'),
    keras.layers.MaxPool2D(),
    keras.layers.Conv2D(filters = 128, kernel_size = (2,2), activation = 'relu'),
    keras.layers.MaxPool2D(),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation = 'relu'),
    keras.layers.Dense(32, activation = 'relu'),
    keras.layers.Dense(1, activation = 'sigmoid')                             
])
model.summary()
from keras.metrics import Recall,AUC
from keras.utils.vis_utils import plot_model
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy',Recall(),AUC()])
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
early_stoppping = EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)
reduce_lr_on_plateau = ReduceLROnPlateau(monitor='val_loss',factor=0.1,patience=5)

model.fit(x=train_gen,batch_size=32,epochs=15,validation_data=val_gen,callbacks=[early_stoppping,reduce_lr_on_plateau])
history= model.history.history
px.line(history,title = "Metrics Plot")
eval_list = model.evaluate(val_gen,return_dict=True)
for metric in eval_list.keys():
    print(metric+f": {eval_list[metric]:.2f}")

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
from keras.applications.xception import Xception
from keras.models import Model
from keras.layers import Dropout

xception = Xception(include_top = False,input_shape = (256,256,3))
input_to_model = xception.input
#turn off training
xception.trainable = False #freeze the parameters of the xception model

#trained part (transfer-learning)
x = Flatten()(xception.output)
x = Dense(64,activation = 'relu')(x)
output_to_model = Dense(1,activation = 'sigmoid')(x) #binary
model2 = Model(inputs = input_to_model,outputs = output_to_model)

model2.compile(optimizer = 'adam',loss = 'binary_crossentropy',metrics = ['accuracy',Recall(),AUC()])
history2 = model2.fit(x = train_gen,batch_size=32,epochs=15,callbacks = [early_stoppping,reduce_lr_on_plateau],validation_data = val_gen)
px.line(history,title='Metrics Plot')

eval_list = model.evaluate(val_gen,return_dict=True)
for metric in eval_list.keys():
    print(metric+f": {eval_list[metric]:.2f}")

#Import other fire images and display them
from PIL import Image
from skimage.transform import resize
img_path = []
for dirpath, dirnames, filenames in os.walk('/content/drive/MyDrive/Fire-Detection/1'):
  for filename in filenames:
    img_path.append(os.path.join(dirpath,filename)) 
print(img_path)
print((img_path[0]))
fig, ax = plt.subplots(11,11,figsize=(15, 15))
ax= ax.ravel()
test_img= []
for i, img in enumerate(img_path):
  _ = Image.open(img)
  _ = np.array(_)
  _ = resize(_, (256, 256, 3))
  test_img.append(_)
  ax[i].axes.xaxis.set_visible(False)
  ax[i].axes.yaxis.set_visible(False)
  ax[i].imshow(_)

#Prediction
test_img = np.array(test_img)
pred = model2.predict(test_img)
plt.plot(pred)
print(pred)

In [ ]:
from PIL import Image
from skimage.transform import resize
img_path = []
for dirpath, dirnames, filenames in os.walk('/content/drive/MyDrive/Fire-Detection/1'):
  for filename in filenames:
    img_path.append(os.path.join(dirpath,filename))
cnt = 0
for dirpath, dirnames, filenames in os.walk('/content/drive/MyDrive/Fire-Detection/0'):
    if cnt>=90 : break
    cnt+=1
    for filename in filenames:
      img_path.append(os.path.join(dirpath,filename)) 
print(img_path)
print((img_path[0]))
fig, ax = plt.subplots(20,10,figsize=(15, 15))
ax= ax.ravel()
test_img= []
for i, img in enumerate(img_path):
  _ = Image.open(img)
  _ = np.array(_)
  _ = resize(_, (256, 256, 3))
  test_img.append(_)
  ax[i].axes.xaxis.set_visible(False)
  ax[i].axes.yaxis.set_visible(False)
  ax[i].imshow(_)

In [ ]:
!curl https://static01.nyt.com/images/2021/02/19/world/19storm-briefing-texas-fire/19storm-briefing-texas-fire-articleLarge.jpg --output predict.jpg
from keras.preprocessing import image
import tensorflow as tf
img = Image.open('predict.jpg')
img = image.img_to_array(img)/255
img = tf.image.resize(img,(256,256))
img = tf.expand_dims(img,axis=0)

print("Image Shape",img.shape)
prediction = int(tf.round(model2.predict(x=img)).numpy()[0][0])
print("The predicted value is: ",prediction,"and the predicted label is:",class_indices[prediction])

In [ ]:
pred = tf.round(model2.predict(test_img))
plt.plot(pred)
print(pred)